In [1]:
import sys
import dgl
import dgl.function as fn
import os
import multiprocessing as mp
from tqdm import tqdm
import pdb
import numpy as np
import torch
import torch.nn as nn
import logging
from utils.parser import parse_args
from utils.dataloader import Dataloader
from utils.utils import config, construct_negative_graph, choose_model, load_mf_model, NegativeGraph
from utils.tester import Tester
from models.sampler import NegativeSampler
import wandb
import math

In [34]:
numbers=[8,4,2,2,2,2,1,1]
square_sum=math.sqrt(sum(i**2 for i in numbers))
numbers = [x / square_sum for x in numbers]

ma=1.2*max(numbers)
mi=0.3*min(numbers)

result=[(x-mi) /(ma-mi) for x in numbers]


result = [x / sum(result) for x in result]

In [64]:
numbers=[8,4,2,2,2,2,1,1]
square_sum=math.sqrt(sum(i**2 for i in numbers))
numbers = [math.exp(x/square_sum) for x in numbers]
ma=max(numbers)
mi=0.2*min(numbers)
result=[(x-mi) /(ma-mi) for x in numbers]
result = [x / sum(result) for x in result]
result=[int(x*20) for x in result]


In [65]:
result

[4, 2, 2, 2, 2, 2, 1, 1]

In [2]:
def pairwise_cosine_similarity(x, y, zero_diagonal: bool = False):
    r"""
    Calculates the pairwise cosine similarity matrix

    Args:
        x: tensor of shape ``(batch_size, M, d)``
        y: tensor of shape ``(batch_size, N, d)``
        zero_diagonal: determines if the diagonal of the distance matrix should be set to zero

    Returns:
        A tensor of shape ``(batch_size, M, N)``
    """
    x_norm = torch.linalg.norm(x, dim=2, keepdim=True)
    y_norm = torch.linalg.norm(y, dim=2, keepdim=True)
    distance = torch.matmul(torch.div(x, x_norm), torch.div(y, y_norm).permute(0, 2, 1))
    

    return distance

In [13]:
x=torch.randn(2,4,32)
y=x
z=pairwise_cosine_similarity(x,y)

In [14]:
print(x)

tensor([[[-5.8162e-01, -2.6666e-01,  1.0256e+00,  8.9986e-01,  2.7631e-01,
          -7.9958e-01,  9.9497e-01,  3.3882e+00,  1.3090e+00,  6.6969e-01,
           1.7313e-01, -1.2706e+00,  2.5242e-02,  2.9491e-01, -3.2103e-02,
          -4.5134e-01,  1.5261e+00,  1.3115e+00,  8.1811e-01, -1.3548e+00,
           1.8962e-01, -1.0528e+00,  6.7586e-01, -1.2010e+00, -2.2413e-01,
           9.6009e-01, -3.8757e-01, -8.1501e-02, -1.4957e+00,  6.4385e-01,
           2.4550e+00, -3.3716e-01],
         [-8.3052e-01,  1.0879e+00, -1.0570e+00, -1.1876e+00,  2.2608e+00,
          -1.2130e-01,  1.5487e+00, -8.5721e-01, -6.5706e-01,  2.0620e+00,
          -1.3625e+00, -7.1285e-01,  1.5581e-02, -1.7115e+00,  1.0161e+00,
          -4.9889e-01,  1.8547e-02,  6.9482e-01,  3.3283e-01, -1.0374e+00,
          -3.0869e-01,  2.6472e+00, -1.0618e+00,  6.7666e-01, -8.2017e-01,
          -4.4604e-01, -5.7727e-01, -1.6490e-01,  9.4377e-02,  1.0181e+00,
          -9.3273e-01,  2.0615e+00],
         [ 4.8081e-01,  3.

In [16]:
print(z.shape)
print(z.mean(dim=2).mean(dim=1).unsqueeze(dim=1))
print(z.mean(dim=2))

torch.Size([2, 4, 4])
tensor([[0.1581],
        [0.2868]])
tensor([[0.1371, 0.1553, 0.1830, 0.1571],
        [0.2358, 0.3025, 0.3012, 0.3075]])
